In [1]:
import CNN2Head_input2 as CNN2Head_input
import os
import tensorflow as tf
import numpy as np
import BKNetStyle2 as BKNetStyle
from const import *

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
''' PREPARE DATA '''
''' PREPARE DATA '''
smile_train, smile_test = CNN2Head_input.getSmileImage()
gender_train, gender_test = CNN2Head_input.getGenderImage()
age_train, age_test = CNN2Head_input.getAgeImage()
'''-------------------------------------
-------------------------------------------------------'''

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  90000
---------------------------------------------------------------
Load age image...................
Done !
Number of age train data:  90000
---------------------------------------------------------------


'-------------------------------------\n-------------------------------------------------------'

In [3]:
print(len(smile_test))
print(len(gender_test))
print(len(age_test))

654
23457
23457


In [4]:
def one_hot(index, num_classes):
    assert index < num_classes and index >= 0
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

In [5]:
sess = tf.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()
x, y_, mask = BKNetStyle.Input()
y_smile_conv, y_gender_conv, y_age_conv, phase_train, keep_prob = BKNetStyle.BKNetModel(x)

Instructions for updating:
Please switch to tf.train.get_or_create_global_step


In [6]:
smile_loss, gender_loss, age_loss, l2_loss, loss = BKNetStyle.selective_loss(y_smile_conv, y_gender_conv,
                                                                             y_age_conv, y_, mask)
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
train_step = BKNetStyle.train_op(loss, global_step)

smile_mask = tf.get_collection('smile_mask')[0]
gender_mask = tf.get_collection('gender_mask')[0]
age_mask = tf.get_collection('age_mask')[0]

y_smile = tf.get_collection('y_smile')[0]
y_gender = tf.get_collection('y_gender')[0]
y_age = tf.get_collection('y_age')[0]

smile_correct_prediction = tf.equal(tf.argmax(y_smile_conv, 1), tf.argmax(y_smile, 1))
gender_correct_prediction = tf.equal(tf.argmax(y_gender_conv, 1), tf.argmax(y_gender, 1))
# age_correct_prediction = tf.equal(tf.argmax(y_age_conv, 1), tf.argmax(y_age, 1))

smile_true_pred = tf.reduce_sum(tf.cast(smile_correct_prediction, dtype=tf.float32) * smile_mask)
gender_true_pred = tf.reduce_sum(tf.cast(gender_correct_prediction, dtype=tf.float32) * gender_mask)
# age_true_pred = tf.reduce_sum(tf.cast(age_correct_prediction, dtype=tf.float32) * age_mask)
age_mae, update_op = tf.metrics.mean_absolute_error(
    tf.argmax(y_age, 1), tf.argmax(y_age_conv, 1), name="age_mae")
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="age_mae")
running_vars_initializer = tf.variables_initializer(var_list=running_vars)
sess.run(running_vars_initializer)

train_data = []
# Mask: Smile -> 0, Gender -> 1, Age -> 2
for i in range(len(smile_train) * 10):
    img = (smile_train[i % 3000][0] - 128) / 255.0
    label = smile_train[i % 3000][1]
    train_data.append((img, one_hot(label, 101), 0.0))
for i in range(len(gender_train)):
    img = (gender_train[i][0] - 128) / 255.0
    label = (int)(gender_train[i][1])
    train_data.append((img, one_hot(label, 101), 1.0))
for i in range(len(age_train)):
    img = (age_train[i][0] - 128) / 255.0
    label = (int)(age_train[i][1])
    train_data.append((img, one_hot(label, 101), 2.0))

saver = tf.train.Saver()

if not os.path.isfile(SAVE_FOLDER2 + 'model-age101.ckpt.index'):
    print('Create new model')
    sess.run(tf.global_variables_initializer())
    print('OK')
else:
    print('Restoring existed model')
    saver.restore(sess, SAVE_FOLDER2 + 'model-age101.ckpt')
    print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary2/", graph=tf.get_default_graph())

learning_rate = tf.get_collection('learning_rate')[0]
current_epoch = (int)(global_step.eval() / (len(train_data) // BATCH_SIZE))
for epoch in range(current_epoch + 1, NUM_EPOCHS):
    print('Epoch:', str(epoch))
    np.random.shuffle(train_data)
    train_img = []
    train_label = []
    train_mask = []

    for i in range(len(train_data)):
        train_img.append(train_data[i][0])
        train_label.append(train_data[i][1])
        train_mask.append(train_data[i][2])

    number_batch = len(train_data) // BATCH_SIZE

    avg_ttl = []
    avg_rgl = []
    avg_smile_loss = []
    avg_gender_loss = []
    avg_age_loss = []

    smile_nb_true_pred = 0
    gender_nb_true_pred = 0
    age_nb_true_pred = 0

    smile_nb_train = 0
    gender_nb_train = 0
    age_nb_train = 0

    print("Learning rate: %f" % learning_rate.eval())
#     for batch in range(number_batch):
    for batch in range(number_batch):
#         if (batch+1)%100==0:
#         print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
        top = batch * BATCH_SIZE
        bot = min((batch + 1) * BATCH_SIZE, len(train_data))
        batch_img = np.asarray(train_img[top:bot])
        batch_label = np.asarray(train_label[top:bot])
        batch_mask = np.asarray(train_mask[top:bot])

        for i in range(BATCH_SIZE):
            if batch_mask[i] == 0.0:
                smile_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    gender_nb_train += 1                        
                else:
                    age_nb_train += 1

        batch_img = CNN2Head_input.augmentation(batch_img, 48)

        ttl, sml, gel, agl, l2l, _ = sess.run([loss, smile_loss, gender_loss, age_loss, l2_loss, train_step],
                                              feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                         phase_train: True,
                                                         keep_prob: 0.5})

        smile_nb_true_pred += sess.run(smile_true_pred, feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                   phase_train: True,
                                                                   keep_prob: 0.5})

        gender_nb_true_pred += sess.run(gender_true_pred,
                                        feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                   phase_train: True,
                                                   keep_prob: 0.5})

#         age_nb_true_pred += sess.run(age_true_pred,
#                                          feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
#                                                     phase_train: True,
#                                                     keep_prob: 0.5})

        sess.run(update_op,
            feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                       phase_train: False,
                       keep_prob: 1})

        '''--------------------------------------- DEBUG -----------------------------------------------------'''
        '''
        sm_mask, em_mask, ge_mask = sess.run([smile_mask, gender_mask, age_mask],
                                             feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                        phase_train: True,
                                                        keep_prob: 0.5})
        print('Smile mask: ', sm_mask)
        print('Gender mask', ge_mask)
        print('Age mask', ag_mask)
        print('Batch mask', batch_mask)

        y_true_sm, y_true_ge, y_true_ag = sess.run([y_smile, y_gender, y_age],
                                                   feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                              phase_train: True,
                                                              keep_prob: 0.5})
        print('Smile label', y_true_sm)
        print('Gender label', y_true_ge)
        print('Age label', y_true_ag)
        print('Batch label', batch_label)

        y_conv_sm, y_conv_ge, y_conv_ag = sess.run([y_smile_conv, y_gender_conv, y_age_conv],
                                                   feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                              phase_train: True,
                                                              keep_prob: 0.5})
        print('Smile conv', y_conv_sm)
        print('Gender conv', y_conv_ge)
        print('Age conv', y_conv_ag)

        '''
        '''---------------------------------- END OF DEBUG ----------------------------------------------------'''

        avg_ttl.append(ttl)
        avg_smile_loss.append(sml)
        avg_gender_loss.append(gel)
        avg_age_loss.append(agl)

        avg_rgl.append(l2l)

    smile_train_accuracy = smile_nb_true_pred * 1.0 / smile_nb_train
    gender_train_accuracy = gender_nb_true_pred * 1.0 / gender_nb_train
    age_train_accuracy = age_nb_true_pred * 1.0 / age_nb_train

    avg_smile_loss = np.average(avg_smile_loss)
    avg_gender_loss = np.average(avg_gender_loss)
    avg_age_loss = np.average(avg_age_loss)

    avg_rgl = np.average(avg_rgl)
    avg_ttl = np.average(avg_ttl)

    summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
    writer.add_summary(summary, global_step=epoch)

    with open('log2.csv', 'a') as f:
        # epochs, smile_train_accuracy, gender_train_accuracy, age_train_accuracy,
        # avg_smile_loss, avg_gender_loss, avg_age_loss, avg_ttl, avg_rgl
        f.write('{0},{1},{2},{3},{4},{5},{6},{7},{8}\n'.format(current_epoch, smile_train_accuracy, gender_train_accuracy, age_train_accuracy, avg_smile_loss, avg_gender_loss, avg_age_loss, avg_ttl, avg_rgl))


    print('\n')

    print('Smile task train accuracy: ' + str(smile_train_accuracy * 100))
    print('Gender task train accuracy: ' + str(gender_train_accuracy * 100))
    age_train_error = sess.run(age_mae)
    print('Age task train error: ' + str(age_train_error))

    print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
    print('Smile loss: ' + str(avg_smile_loss))
    print('Gender loss: ' + str(avg_gender_loss))
    print('Age loss: ' + str(avg_age_loss))

    saver.save(sess, SAVE_FOLDER2 + 'model-age101.ckpt')

Default GPU Device: /device:GPU:0
Restoring existed model
INFO:tensorflow:Restoring parameters from ./save/current2/model-age101.ckpt
OK
Epoch: 2
Learning rate: 0.009500


Smile task train accuracy: 92.40160106737825
Gender task train accuracy: 86.98981860216965
Age task train error: 23.80281
Total loss: 5.0612297. L2-loss: 0.45837152
Smile loss: 0.30672523
Gender loss: 0.3693332
Age loss: 3.9267995
Epoch: 3
Learning rate: 0.008574


Smile task train accuracy: 93.01627534685166
Gender task train accuracy: 87.78340890904242
Age task train error: 23.847315
Total loss: 4.876975. L2-loss: 0.34247923
Smile loss: 0.28861922
Gender loss: 0.35155982
Age loss: 3.894317
Epoch: 4
Learning rate: 0.008145


Smile task train accuracy: 93.21282059833906
Gender task train accuracy: 88.11498188123875
Age task train error: 23.859528
Total loss: 4.823342. L2-loss: 0.34222355
Smile loss: 0.27414915
Gender loss: 0.34088728
Age loss: 3.8660817
Epoch: 5
Learning rate: 0.007351


Smile task train accuracy: 93

Epoch: 33
Learning rate: 0.000694


Smile task train accuracy: 99.80658930238762
Gender task train accuracy: 90.63726198548292
Age task train error: 23.555653
Total loss: 4.176133. L2-loss: 0.30121937
Smile loss: 0.04760239
Gender loss: 0.26163435
Age loss: 3.5656774
Epoch: 34
Learning rate: 0.000627


Smile task train accuracy: 99.87997199346515
Gender task train accuracy: 90.63514294606732
Age task train error: 23.547626
Total loss: 4.1602077. L2-loss: 0.29787642
Smile loss: 0.043504372
Gender loss: 0.2604227
Age loss: 3.5584044
Epoch: 35
Learning rate: 0.000595


Smile task train accuracy: 99.91664165916441
Gender task train accuracy: 90.76669964538613
Age task train error: 23.53937
Total loss: 4.135363. L2-loss: 0.29260865
Smile loss: 0.034236368
Gender loss: 0.25792393
Age loss: 3.5505943
Epoch: 36
Learning rate: 0.000537


Smile task train accuracy: 99.89994997498749
Gender task train accuracy: 90.76469934422585
Age task train error: 23.5318
Total loss: 4.1202536. L2-loss: 0.2865

KeyboardInterrupt: 